<h2> Text Mining with Cosine Similarities </h2>
<h4> Ian Russell </h4>
<p> To begin, the document must be processed (a). In order to do so, the following six steps need be performed. [1] Convert to lower case, [2] remove stop words, [3] remove punctuation, [4] remove singular characters, [5] stem all words, and [6] replace digits with their english representation. </p>

In [5]:
import re
import os
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import pandas as pd
from num2words import num2words
import ast
import csv
import numpy as np
import queue
import time
import json

<h2> Building a Tool Set </h2>
<h3> Processing documents </h3>
<p> In order to process many documents in a clean and readable fashion, each document will be treated as an object. This document class is shown below and incorporates a pre-process functionality with the rules mentioned above. Furthermore, there is a similar pre-process function to be called on pure text as opposed to a file.</p>

In [6]:
class Document:
    
    def __init__(self, filename = '', path = '', text = ''):
        self.filename = filename
        self.path = path
        self.text = text
        
    def process_query(self):
        
        
        stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
        porter = PorterStemmer()
        
        query = self.text
        
        # Tokenize
        query = re.sub(r'\W+', ' ', query).split()
        # apply lower case 
        query = [x.lower() for x in query]
        # remove stop words
        query = [i for i in query if not i in stop_words]
        # remove single characters
        query = [i for i in query if len(i) > 1]
        # stemming
        tmp = []
        for word in query:
            tmp.append(porter.stem(word))
        query = tmp

        # convert numbers
        index = 0
        for word in query:
            if word.isdigit():
                query[index] = num2words(word)
            index += 1

        return query

        
    def pre_process(self):
        """
        Pre-processor function to remove and tokenize indivual document
        object for prepartion of analysis. Returns processed data.
        """
        
        stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
        porter = PorterStemmer()
        
        with open(os.path.join(path, self.filename), encoding="utf8", errors="surrogateescape") as text:

            data = text.read()
            
            # Tokenize
            data = re.sub(r'\W+', ' ', data).split()
            # apply lower case 
            data = [x.lower() for x in data]
            # remove stop words
            data = [i for i in data if not i in stop_words]
            # remove single characters
            data = [i for i in data if len(i) > 1]
            # stemming
            tmp = []
            for word in data:
                tmp.append(porter.stem(word))
            data = tmp
            
            # convert numbers
            index = 0
            for word in data:
                if word.isdigit():
                    data[index] = num2words(word)
                index += 1
            
            print("Document ready!")
            
            return data

<h3> Calculating TF-IDF Vectors </h3>
<p> Simlar to the document class, the TFIDFVector class will generate a vector object from data parameters such as document frequencies and universal word totals. This object maintains the ability to convert itself into an TF-IDF vector coressponding to an individual document. Similar to the case in the document class a seperate vectorize function is used for text based queries.
</p>

<p> It is useful to calculate such a vector that includes both term frequecy and inverse document frequency because the information contained by those terms encompasses statistics for the indivual document and the entire corpus, respectively. </p>

In [7]:
class TFIDFVector:
    def __init__(self, document, N, DF):
        """
        Creates document vector to compute TF-IDF.
        For document vectorization, document should
        be a dictionary with word counts. For a query
        document should be in a string format.
        """
        self.document = document
        self.N = N
        self.DF = DF
        
        
    def vectorize(self):
        """
        Function generates TF-IDF Vector for
        individual document in corpus.
        """
        tf_idf_vector = {}
        for term in self.document:
            # term frequency
            count = self.document[term]
            tf = count/len(self.document)
            # document frequency
            df = sum(self.DF[term])
            # inverse document frequency
            idf = np.log(self.N/(df + 1))
            # TF-IDF
            tf_idf = tf*idf
            save = {term : tf_idf}
            tf_idf_vector.update(save)
            
        return tf_idf_vector
    
    def query_vectorize(self):
        
        tf_idf_vector_query = {}
       
        for term in self.document:
            count = counter(self.document, term)[1]
            tf = count/len(self.document)
            # document frequency
            df = sum(self.DF[term])
            # inverse document frequency
            idf = np.log(self.N/(df + 1))
            # TF-IDF
            tf_idf = tf*idf
            save = {term : tf_idf}
            tf_idf_vector_query.update(save)
            
        return tf_idf_vector_query
    


<h3> Utility Functions </h3> 
<p> Each function below is used to help calculate individual data parameters.
the precise purpose of each function is described in its documentation. </p>

In [15]:
def main(directory):
    """
    Main driver function, takes in directory and returns term counts for
    each individual document.
    """
    docs = []
    for entry in entries:
        docs.append(Document(entry, path))

    processed = []
    for document in docs:
         processed.append(document.pre_process())
    
    processed_counts = termCounts(processed)
    
    with open('wordCounts.txt', 'w') as file:
        file.write(json.dumps(processed_counts))
    
    return processed_counts


def counter(lst, term): 
    """
    Utility function to count redundant occurences
    in a list.
    """
    count = 0
    for ele in lst: 
        if (ele == term): 
            count = count + 1
    return term, count


def termCounts(corpus):
    """
    Takes list of pre-processed documents (corpus) and returns individual
    term counts for each document.
    """
    count = 0
    termCount_corpus = []
    for document in corpus:
        termCounts_doc = {}
        tmp = []
        print("Counting... " + str(count))
        for term in document:
            tmp.append(counter(document,term))
        terms = set(tmp)
        terms = list(terms)
        termCounts_doc.update(terms)
        termCount_corpus.append(termCounts_doc)
        print(len(termCount_corpus))
        count += 1
    return termCount_corpus

    


def universe_size(data):
    """
    Utility function to compute and return
    term universe size 'N'.
    """
    N = 0
    for doc in data: 
        n=0
        for term in doc:
            count = doc[term]
            n += count
        N += n
    return N
        
def document_frequency(data):
    """
    Utility function to compute document frequency for 
    all terms in pre-processed corpus.
    """
    DF = {}
    for i in range(len(data)):
        tokens = data[i]
        for w in tokens:
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
    return DF

<h3> Cosine Similarity </h3>
<p> Cosine similarity allows for a metric to compare to n-dimensional vectors landing between the values of 0 and 1. This similarity is quantified by the tightness of the angle between any two vectors. The function below performs this calculation for every possible pair of documents in the data set (250 x 250)!

In [15]:
def cosineSim(data):
    """
    Function to compute the pairwise cosine similarity
    throughout all documents. Produces an n x n matrix
    saved as a csv. n is number of documents. 
    
    """
    
    start_time = time.time()
    
    # Initialize all tf-idf vectors
    start_vectors  = []
    print('Initializing...')
    print()
    
    # Append each tfd-idf vector to start_vectors (all docs)
    for i in range(len(data)):
        start_vectors.append(TFIDFVector(data[i], universe_size(data), document_frequency(data)).vectorize())

    # Queue up vectors for comparison
    to_compare = queue.Queue(maxsize = len(data))
    cols = ["col" + str(i) for i in range(len(data))]

    for vector in start_vectors:
        to_compare.put(vector)

    print("Vectors ready")
    print()
    print("Working...")
    tick = 0
    
    # Start comparisons
    sims = pd.DataFrame()
    
    # Outer loop handles queued vectors
    for vector in start_vectors:
        base = to_compare.get()
        print()
        column = []
        
        # Inner loop compares each vector in data to current vector in the queue
        for i in range(len(data)):
            
            # Convert pair of vectors to data frame
            comparison = pd.DataFrame([base, start_vectors[i]])
            
            # Replace non-present words from opposing document with 0 values
            comparison.fillna(0, inplace = True)
            a = comparison.iloc[0]
            b = comparison.iloc[1]
            
            # Convert to numpy vectors
            a = a.to_numpy()
            b = b.to_numpy()

            # manually compute cosine similarity
            dot = np.dot(a,b)
            norma = np.linalg.norm(a)
            normb = np.linalg.norm(b)
            cos = round(dot / (norma * normb), 5)
            column.append(cos)
        
        # Insert pair wise comparison as column in final matrix
        insertion = pd.Series(column)
        sims.insert(tick, cols[tick], insertion)




        tick += 1
        print("#####################################")
        print("Computation complete for vector " + str(tick) + "/" + str(len(data)))
        print("#####################################")



    export_csv = sims.to_csv (r'/home/ian/Dropbox/School/Current_courses/Data_Mining/assignment_3/matrix.csv', index = None, header=True)

    print("--- %s minutes ---" % round(((time.time() - start_time)/60), 2))
    
    

In [15]:
def retrieval(queries, data):
    """
    Function performs a comparison
    analysis between TF-IDF vectorized queries 
    and all documents in the data parameter. Data
    must be in pre-processed format.
    """
    
    start_time = time.time()
    
    # Initialize all tf-idf vectors
    print()
    print('Initializing data...')
    
    start_vectors  = []
    
    # Append each tfd-idf vector to start_vectors (all docs)
    for i in range(len(data)):
        start_vectors.append(TFIDFVector(data[i], universe_size(data), document_frequency(data)).vectorize())

    # Queue up vectors for comparison
    to_compare = queue.Queue(maxsize = len(data))
    cols = ["col" + str(i) for i in range(len(data))]

    for vector in start_vectors:
        to_compare.put(vector)
    
    print('Intializing queries...')
    
    # Pre-process queries
    processed_queries = []
    for query in queries:
        filename = ''
        q = Document(filename, path, text = query)
        processed_queries.append(q.process_query())
    
    
    # Vectorize queries
    query_vectors = []
    for i in range(len(queries)):
        query_vectors.append(TFIDFVector(processed_queries[i], universe_size(data), document_frequency(data)).query_vectorize())
    
    print('Queries processed!')
    
    # Comparisons
    
    
    # Queue up vectors for comparison
    to_search = queue.Queue(maxsize = len(query_vectors))
    cols = ["col" + str(i) for i in range(len(data))]

    for vector in query_vectors:
        to_search.put(vector)
    
    retrievals = pd.DataFrame()
    
    tick = 0
    # Outer loop handles queued vectors
    
    print("Searching...")
    for vector in query_vectors:
        base = to_search.get()
        column_retrievals = []
        
        # Inner loop compares each vector in data to current vector in the queue
        for i in range(len(data)):
            
            # Convert pair of vectors to data frame
            comparison = pd.DataFrame([base, start_vectors[i]])
            
            # Replace non-present words from opposing document with 0 values
            comparison.fillna(0, inplace = True)
            a = comparison.iloc[0]
            b = comparison.iloc[1]
            
            # Convert to numpy vectors
            a = a.to_numpy()
            b = b.to_numpy()

            # manually compute cosine similarity
            dot = np.dot(a,b)
            norma = np.linalg.norm(a)
            normb = np.linalg.norm(b)
            cos = round(dot / (norma * normb), 5)
            column_retrievals.append(cos)
        
        # Insert pair wise comparison as column in final matrix
        insertion = pd.Series(column_retrievals)
        retrievals.insert(tick, cols[tick], insertion)
        tick += 1
    print('Done!')
    print("--- %s seconds ---" % round(((time.time() - start_time)), 2))
        
    return retrievals
    

    

<h3> Pre-processing </h3>

<p>
This section of code combines the above functions
and classes to generate the necessary term counts
and document frequencies to compute individual TF-IDF
vectors for idividual documents. Run to process new 
data. Data is saved to file 'termCounts.txt'.
</p>


In [6]:
#####################
#### PRE-PROCESS ####
#####################

# Directory path
path = '/home/ian/Dropbox/School/Current_courses/Data_Mining/assignment_3/Data'

# Data directory
entries = os.listdir(path)

# Uncomment if file not already created
results = main(entries)


In [38]:
################
### ANALYSIS ###
################

with open('wordCounts.txt', 'r') as file:
    data = file.read()
    data = ast.literal_eval(data)


data;

In [18]:
df = pd.read_csv('matrix.csv')
df
# Run cosineSim() if not been run already (run time ~ 75 minutes)
# matrix = cosineSim(data)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col239,col240,col241,col242,col243,col244,col245,col246,col247,col248
0,1.00000,0.13074,0.15071,0.05337,0.07692,0.05715,0.03246,0.13397,0.07524,0.09731,...,0.15486,0.12591,0.09221,0.11318,0.02616,0.13777,0.00555,0.10723,0.05787,0.10710
1,0.13074,1.00000,0.42906,0.10437,0.13335,0.15244,0.06316,0.33175,0.12327,0.16180,...,0.50410,0.20103,0.13672,0.14929,0.06043,0.13403,0.24070,0.21832,0.11449,0.18694
2,0.15071,0.42906,1.00000,0.14807,0.17398,0.15496,0.07850,0.45667,0.16531,0.18229,...,0.54739,0.23365,0.16000,0.18468,0.07802,0.14582,0.21752,0.25073,0.13587,0.23150
3,0.05337,0.10437,0.14807,1.00000,0.06599,0.06573,0.03951,0.15739,0.08372,0.08062,...,0.13060,0.09218,0.05921,0.07375,0.03756,0.04925,0.01871,0.09496,0.06481,0.08932
4,0.07692,0.13335,0.17398,0.06599,1.00000,0.07362,0.03496,0.19353,0.10675,0.08951,...,0.16040,0.10332,0.08538,0.05690,0.03659,0.05757,0.01401,0.11778,0.11069,0.13752
5,0.05715,0.15244,0.15496,0.06573,0.07362,1.00000,0.04047,0.17547,0.06948,0.08303,...,0.15040,0.12113,0.07376,0.07731,0.04388,0.06658,0.02480,0.10961,0.06009,0.09927
6,0.03246,0.06316,0.07850,0.03951,0.03496,0.04047,1.00000,0.09304,0.04752,0.04191,...,0.08173,0.08749,0.04751,0.04556,0.03665,0.05219,0.01117,0.05418,0.04220,0.05506
7,0.13397,0.33175,0.45667,0.15739,0.19353,0.17547,0.09304,1.00000,0.19088,0.21597,...,0.43341,0.25874,0.17991,0.17737,0.11308,0.15914,0.03475,0.26929,0.18833,0.28217
8,0.07524,0.12327,0.16531,0.08372,0.10675,0.06948,0.04752,0.19088,1.00000,0.09629,...,0.16176,0.17275,0.08042,0.12434,0.04587,0.09731,0.01108,0.16015,0.11140,0.13064
9,0.09731,0.16180,0.18229,0.08062,0.08951,0.08303,0.04191,0.21597,0.09629,1.00000,...,0.20069,0.15127,0.10650,0.09937,0.05922,0.09822,0.00917,0.16614,0.10291,0.16906


<h3> Query Processing </h3>

<p> In order to search the data set and find a best matching document to some query, the query itself must be 'vectorized' (i.e. compute a TF-IDF vector) and that vector must be compared via the cosine similarity to all other documents in the data set. Once completed the results for each query are sorted by cosine similarity output from largest to smallest. We the take the top 10 results and display them below.

In [19]:

queries = ["""Once upon a time . . . 
there were three little pigs, 
who left their mummy and daddy to see the
world.""",
"""There once lived a poor tailor, who had a son called Aladdin, 
a careless, idle boy who would
do nothing but play all day long 
in the streets with little idle boys like himself."""]



In [39]:
df = retrieval(queries, data)


Initializing data...
Intializing queries...
Queries processed!
Searching...
Done!
--- 45.13 seconds ---


In [53]:
df["Document"] = entries

df['query1'] = df["col0"]
df['query2'] = df["col1"]
df.drop(['col0', 'col1'], axis=1)

query1 = pd.DataFrame(df['Document'])
query1['Score'] = df['query1']

query2 = pd.DataFrame(df['Document'])
query2['Score'] = df['query2']

five_d1 = query1.sort_values(by=['Score'], ascending=False).head(n=10)
five_d2 = query2.sort_values(by=['Score'], ascending=False).head(n=10)

<h3> Results </h3>
<p> The resultant scores for each query given are displayed in the data frames below.
As can be seen, the query returned the most relavent documents by highest score. The validity of this result can be checked by observing the titles of the highest scoring documents!</p>

In [54]:
# Query 1
five_d1

,Document,Score
43,3lpigs.txt,0.40721
238,enginer.txt,0.14670
7,gulliver.txt,0.14432
27,lgoldbrd.txt,0.14432
160,empty.txt,0.13223
81,ccm.txt,0.13131
2,5orange.txt,0.12428
178,hound-b.txt,0.11972
197,darkness.txt,0.11508
70,abbey.txt,0.11485


In [55]:
# Query 2
five_d2

,Document,Score
97,alad10.txt,0.35603
28,adv_alad.txt,0.33792
116,tinsoldr.txt,0.16191
147,yukon.txt,0.12163
202,fantasy.txt,0.12065
197,darkness.txt,0.10851
160,empty.txt,0.10630
236,snowmaid.txt,0.10380
146,lmtchgrl.txt,0.10281
53,aesop11.txt,0.09160
